DEVELOPER SURVEY ANALYSIS - NOTEBOOK


OUTLINE

In [1]:
# import data analysis & graph packages
import pandas as pd
import numpy as np

import zipfile
import glob
import sys
import os
import re

import requests
from io import BytesIO
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
# import survey data

# data directory
folder_name = 'sof-survey-data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
# download developer survey file
survey_results_url = 'https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2020.zip'
print(survey_results_url)
response = requests.get(survey_results_url)

# open zip file and return readable and writeable view
zipDoc = zipfile.ZipFile(BytesIO(response.content))

# save extracted zip file
zipDoc.extractall(folder_name)


https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2020.zip


In [3]:
# import data
df = pd.read_csv('./sof-survey-data/survey_results_public.csv')
schema = pd.read_csv('./sof-survey-data/survey_results_schema.csv')


In [4]:
# import schema survey questions
schema_fname =  './sof-survey-data/survey_results_schema.csv'
schema_df = pd.read_csv(schema_fname, index_col='Column')
schema = schema_df.QuestionText


**Import helper files**

In [6]:
pwd

'/Users/andrewnachtigal/Documents/00-ComputerScience/03-Udacity/03-Udacity-Data-Science/00-git-repo/Udacity-Data-Science/01-blog-post/jupyter-notebooks'

In [5]:
# import helper files
from helper_functions.helper_functions import print_question_text

# https://stackoverflow.com/questions/4383571/importing-files-from-different-folder

ModuleNotFoundError: No module named 'helper_functions'

In [ ]:
# call function to Display question text for survey questions of interest
question_list = ['DevType', 'JobSat']

for question in question_list:
    print_question_text(question)

Q1. Can we distinguish between data roles and developer roles? 

**What factors are correlated with each group?**


* Individual survey responses may contain any combination of the developer types and therefore contain 
significant variation. 
* For the purpose of analysis, we focus on responses containing either developer or data roles.

Q2. Can we evaluate job satisfaction via descriptive statistics? 

**plus graphical evaluation**


* Individual 
* For 

Q3. Can we classify the best predictors of job satisfaction through a machine learning model?

**Models evaluated**


* Individual 
* For 

### Stackoverflow Survey Data 

**Steps of Data Science Process**  
gather, assess, clean, analyze, model, visualize

### Survey Questions


In [ ]:
# adjsut rows and column display
#pd.set_option('display.max_rows', 100)
#pd.set_option('display.max_colwidth', 200)
#pd.set_option('display.max_columns', 180)

In [ ]:
 schema

In [ ]:
# reset display ... ?



# Q1

## Developer Roles


In [ ]:
# drop NaNs
df = df.dropna(subset=['DevType'], axis=0)


In [ ]:
# unique DevType roles
DevRoles = df.DevType.unique()
DevStr = []
for i in range(len(DevRoles)):
    '''Create list of unique developer roles found in survey responses under 'DevType'.
        Iterate over DevType column
    '''
    role = DevRoles[i].split(";")
    DevStr.append(role)
DevTypes = set(list(chain.from_iterable(DevStr)))
DevTypes


In [ ]:
# specify data and dev role markers
data_roles = ['Data or business analyst', 'Data scientist or machine learning specialist', 'Engineer, data']
dev_roles = ['Developer']


In [ ]:
# subset responses including developer roles
df_dev = df[df['DevType'].str.contains('|'.join(dev_roles), na=False) == True].copy(deep=True)

# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

In [ ]:
# drop responses including data roles
df_dev = df_dev[~df_dev['DevType'].str.contains('|'.join(data_roles), na=False) == True]


In [ ]:
# shape of developer roles dataset
print('dev roles dataframe contains', np.shape(df_dev)[0], 'rows and', np.shape(df_dev)[1], 'columns.')


## Data Roles


In [ ]:
# subset all responses including data
df_data = df[df['DevType'].str.contains('|'.join(data_roles), na=False) == True].copy(deep=True)


In [ ]:
# shape of data roles dataframe
print('data roles dataframe contains', np.shape(df_data)[0], 'rows and', 
      np.shape(df_data)[1], 'columns.')


### Add Indicator Column for developer and data roles: 'is_data_role'

In [ ]:
# set indicator column for data and dev roles
df_data['is_data_role'] = 'data'
df_dev['is_data_role'] = 'dev'


In [ ]:
# join data and developer data
df = pd.concat([df_data, df_dev])

# reset index
df = df.reset_index(drop=True)


In [ ]:
df.head(3)


## Data & Developer Role Feature Exploration

In [ ]:
df.shape


In [ ]:
# column datatypes
pd.set_option('display.max_rows', 10)
display(df.dtypes)


## Data vs. Dev

* explore statistical/visual similarities and differences
* select features manually

Features

* Age
* Converted Comp
* EdLevel


# Age

In [ ]:
# describe Age column

print(df.Age.describe())

# check for null values
print('dataset contains', df['Age'].isna().sum(), 'nan values')


In [ ]:
# unique Age responses
print(list(df.Age.unique()))


## Age Histogram

In [ ]:
# Age histogram
AgeDev = df.loc[df.is_data_role=='dev', 'Age']
AgeData = df.loc[df.is_data_role=='data', 'Age']

kwargs = dict(histtype='stepfilled', alpha=0.3, bins=100)
fig, ax = plt.subplots(figsize=(8, 6))

plt.hist(AgeDev, **kwargs, color='g', label='Dev')
plt.hist(AgeData, **kwargs, color='b', label='Data')
plt.gca().set(ylabel='Frequency', xlabel='Age')
plt.title('Frequency Histogram of Respondent Age', fontsize= 15, fontweight='bold')
plt.xlim(0,70)
plt.legend();

plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"



In [ ]:
# Age: Data mean vs Dev mean 
print('data role mean Age is', round(AgeData.mean(),2))  
print('dev role mean Age is',  round(AgeDev.mean(),2))


# Age Conclusion ->
* **Mean Age is slightly higher for developers.**

# Converted Comp


In [ ]:
# describe Age column
print(df.ConvertedComp.describe())

# check for null values
print('dataset contains', df['ConvertedComp'].isna().sum(), 'nan values')


In [ ]:
# Distribution of Dev and Data Compensation

CompDev = df.loc[df.is_data_role=='dev', 'ConvertedComp'].dropna()
CompData = df.loc[df.is_data_role=='data', 'ConvertedComp'].dropna()

kwargs = dict(histtype='barstacked', alpha=0.3, bins=100)
fig, ax = plt.subplots(figsize=(8, 6))

plt.hist(CompDev, **kwargs, color='g', label='Dev')
plt.hist(CompData, **kwargs, color='b', label='Data')
plt.gca().set(ylabel='Frequency', xlabel='Compensation')
plt.title('Distribution of Respondent Compensation', fontsize= 15, fontweight='bold')
plt.legend();

plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"


In [ ]:
# vertical boxplot
df_z = df[df['ConvertedComp'].notnull()]

sns.set(style="white", palette="Blues", color_codes=True)
plt.figure(figsize=[10, 10])
#df['comp_log'] = np.log(df['ConvertedComp'].values)
#ax = sns.boxplot(x='is_data_role', y='comp_log', data=df)
ax = sns.boxplot(x='is_data_role', y='ConvertedComp', data=df_z)
plt.title('Distribution of Respondent Compensation', fontsize= 15, fontweight='bold')
ax.set_xlabel("Compensation (USD))", labelpad=20, weight='bold', size=12)
ax.set_ylabel(" ", labelpad=20, weight='bold', size=12)
ax.legend(['Data Role', 'Dev Role'])

plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"


In [ ]:
# mean comensation: Data vs Dev
print('data role mean compensation is', round(CompData.mean()))
print('dev role mean compensation is',  round(CompDev.mean()))
print("")
print('Average Data Roles earn more by $', round(CompData.mean()) - round(CompDev.mean()))


# Compensation Conclusion ->
* **Average data roles pay 15% more than Dev roles.**

# Education (EdLevel)

In [ ]:
# plot education levels
educ_order = ['I never completed any formal education',
                'Primary/elementary school',
                'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
                'Associate degree (A.A., A.S., etc.)',
                'Some college/university study without earning a degree',
                'Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
                'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
                'Professional degree (JD, MD, etc.)',
                'Other doctoral degree (Ph.D., Ed.D., etc.)']

# respondent education level
fig, axes = plt.subplots(2, 1, figsize=(10, 10), sharey=True)
fig.suptitle('Respondent Education Level')

plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

# Data
plt.subplot(2, 1, 1)
sns.countplot(ax=axes[0], y=df_data.EdLevel, order=educ_order, palette="ch:.25")
axes[0].set_title('Data')
plt.xticks(rotation = 50);
plt.title(schema['EdLevel'])
plt.xlabel(None);
plt.ylabel('Data');

# Dev
plt.subplot(2, 1, 2)
sns.countplot(ax=axes[1], y=df_dev.EdLevel, order=educ_order, palette="ch:.25")
axes[1].set_title('Dev')
plt.xticks(rotation = 50);
plt.xlabel(None);
plt.ylabel('Dev');

plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"


**Education Level for Data and Dev Roles: Formal Education Ratio**

In [ ]:
# make edlev index key list
edlev_index = ['associate','bachelors', 'no degree', 'masters', 'doctoral', 'elementary', 'professional', 
               'secondary', 'some college']


In [ ]:
# data edlev
data_edlev_val = pd.DataFrame(df.loc[df['is_data_role']=='data']['EdLevel'].value_counts(sort=False, dropna=True))
data_edlev_val.sort_index(inplace=True)
# convert data edlev to dictionary
data_edlev_dict = dict(zip(edlev_index, data_edlev_val['EdLevel'].to_list()))


In [ ]:
# data total
data_edlev_total = data_edlev_val.sum()
# data bachelor's
data_bach_ratio = data_edlev_dict['bachelors'] / data_edlev_total
# data pgrad
data_pgrad_ratio = (sum([data_edlev_dict.get(k) for k in ["masters", "doctoral", 
                                                          "professional"]])) / data_edlev_total


In [ ]:
# dev edlev
dev_edlev_val = pd.DataFrame(df.loc[df['is_data_role']=='dev']['EdLevel'].value_counts(sort=False, dropna=True))
dev_edlev_val.sort_index(inplace=True)
# convert dev edlev to dictionary
dev_edlev_dict = dict(zip(edlev_index, dev_edlev_val['EdLevel'].to_list()))


In [ ]:
# dev total
dev_edlev_total = dev_edlev_val.sum()
# dev bachelor's
dev_bach_ratio = dev_edlev_dict['bachelors'] / dev_edlev_total
# dev pgrad
dev_pgrad_ratio = (sum([dev_edlev_dict.get(k) for k in ["masters", "doctoral", 
                                                        "professional"]])) / dev_edlev_total


In [ ]:
# formal education ratios
print('data bachelors ratio is:', round(data_bach_ratio[0],4))
print('data postgrad ratio is:', round(data_pgrad_ratio[0],4))
print('dev bachelors ratio is:', round(dev_bach_ratio[0],4))
print('dev postgrad ratio is:', round(dev_pgrad_ratio[0],4))


# Education Conclusion ->
* **College degrees are very common among both data and developer roles.**
* **Bachelor's degrees dominate among developers while post-graduate degrees are twice as common for data roles.**

# Q2

# Q3
**Can we classify good predictors of job satisfaction with machine learning?**


**FEATURE MODELING**
* remove unemployed
* method for determining useful features / feature selection
    * trim features by intuition
    * programmaticaly remove uncorrelated columns
* parse feature columns into useful types
* expand columns with list-like structures
* consider distribution of numeric features
* manage missing values
* encode categorical features

In [ ]:
df.head(3)

In [ ]:
# all features
df.columns

# DATA PREPARATION
* drop unemployed responses
* manually drop feature columns that are not of interest
* drop NaN values  
* encode categorical feature columns
* drop string object columns
* reset index


In [ ]:
# drop unemployed responses
df = df[df.Employment.isin(['Employed full-time', 'Independent contractor, freelancer, or self-employed', 
                            'Employed part-time'])]
 
# narrow feature set: drop column features that are not of interest
df = df[['Hobbyist', 'Age', 'Age1stCode','ConvertedComp','EdLevel','JobSat','JobFactors','JobSeek',
         'WorkWeekHrs','YearsCode','YearsCodePro','is_data_role']]


In [ ]:
df.shape

### Model Features

In [ ]:
list(df.columns)

In [ ]:
# column datatypes
pd.set_option('display.max_rows', 75)
display(df.dtypes)


## Categorical Feature Mappings


In [ ]:
# categorical variable responses strings

EdLevel_strs = ['I never completed any formal education',
                'Primary/elementary school',
                'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
                'Associate degree (A.A., A.S., etc.)',
                'Some college/university study without earning a degree',
                'Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
                'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
                'Professional degree (JD, MD, etc.)',
                'Other doctoral degree (Ph.D., Ed.D., etc.)']

JobSeek_strs = ['I am not interested in new job opportunities',
                 'I’m not actively looking, but I am open to new opportunities',
                 'I am actively looking for a job']

JobFactors_strs = ['Diversity of the company or organization',
                 'Family friendliness',
                 'Financial performance or funding status of the company or organization',
                 'Flex time or a flexible schedule',
                 'How widely used or impactful my work output would be',
                 'Industry that I’d be working in',
                 'Languages, frameworks, and other technologies I’d be working with',
                 'Office environment or company culture',
                 'Opportunities for professional development',
                 'Remote work options',
                 'Specific department or team I’d be working on']


# Hobbyist

In [ ]:
# describe data
print(df.Hobbyist.describe())

# check for null values
print('dataset contains', df['Hobbyist'].isna().sum(), 'nan values')


In [ ]:
# 'Hobbyist' responses
print(list(df.Hobbyist.unique()))


In [ ]:
# convert 'Hobbyist' to yes=1 or no=0
df['Hobbyist_bin'] = df.Hobbyist.map(dict(Yes=1, No=0))


In [ ]:
df.Hobbyist_bin.head()


# Age

In [ ]:
# describe age column
print(df.Age.describe())

# check for null values in jobsat column
print('dataset contains', df['Age'].isna().sum(), 'nan values')


In [ ]:
# 'Age' unique responses
print(list(df.Age.unique()))


In [ ]:
# drop NaNs
df = df.dropna(subset=['Age'], axis=0)


In [ ]:
df.shape


# Age1stCode

In [ ]:
# drop nulls Years Coding responses
df = df[df['Age1stCode'].notnull()]


In [ ]:
# replace 'Less than 1 year' with 0; 'More than 50 years' with 51
df['Age1stCode'] = df['Age1stCode'].str.replace('Younger than 5 years', '4')
df['Age1stCode'] = df['Age1stCode'].str.replace('Older than 85', '85')


In [ ]:
# convert 'Age1stCode' to type integer
df['Age1stCode'] = df['Age1stCode'].astype(str).astype(int)


In [ ]:
df.shape


# ConvertedComp


In [ ]:
# drop nulls from ConvertedComp column
df = df[df['ConvertedComp'].notnull()]

df['ConvertedComp'].head()


In [ ]:
df.shape


# EdLevel

In [ ]:
# drop nan job factors
df = df.dropna(subset=['EdLevel'], axis=0)


In [ ]:
# create ordinal encoding for education level

EdLevel_dict = {'I never completed any formal education':1,
               'Primary/elementary school':2,
               'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)':3,
               'Associate degree (A.A., A.S., etc.)': 4,
               'Some college/university study without earning a degree':5,
               'Bachelor’s degree (B.A., B.S., B.Eng., etc.)':6,
               'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)':7,
               'Professional degree (JD, MD, etc.)':8,
               'Other doctoral degree (Ph.D., Ed.D., etc.)':9}


In [ ]:
pwd

In [ ]:
# import helper files
from helper_functions.helper_functions import replace_values_in_string


In [ ]:
# replace values in string function
# https://stackoverflow.com/questions/14156473/can-you-write-a-str-replace-using-dictionary-values-in-python

def replace_values_in_string(text, args_dict):
    for key in args_dict.keys():
        text = text.replace(key, str(args_dict[key]))
    return text


In [ ]:
# encode EdLevel as integer
df['EdLevel_encode'] = replace_values_in_string(df.EdLevel, EdLevel_dict)


In [ ]:
# inspect data
df.EdLevel_encode


In [ ]:
df.shape


# Target Variable: Job Satisfaction 
* simplified binary classification problem
* ordinal classification


In [ ]:
# check for NaNs
df['JobSat'].isna().sum()


In [ ]:
# drop NaNs
df = df.dropna(subset=['JobSat'], axis=0)


In [ ]:
# encode job satisfaction string values to encoded categorical values 
JobSat_dict = {
    'Very satisfied': 5,
    'Slightly satisfied': 4,
    'Neither satisfied nor dissatisfied': 3,
    'Slightly dissatisfied' : 2,
    'Very dissatisfied' : 1
    }


In [ ]:
# encode JobSat
df['JobSat_encode'] = replace_values_in_string(df.JobSat, JobSat_dict)


In [ ]:
df['JobSat_encode'].head()


In [ ]:
#  Encode 'JobSat' feature column as a Binary classification problem:

# 'Very satisfied,'Slightly satisfied' = 1
# 'Neither satisfied nor dissatisfied','Slightly dissatisfied','Very dissatisfied' = 0

JobSat_bin_map = {
    'Very satisfied': 1,
    'Slightly satisfied': 1,
    'Neither satisfied nor dissatisfied': 0,
    'Slightly dissatisfied' : 0,
    'Very dissatisfied' : 0
    }

df['JobSat_encode_bin'] = df['JobSat'].replace(JobSat_bin_map)


In [ ]:
df['JobSat_encode_bin'].head()


In [ ]:
df.shape

# JobFactors

In [ ]:
# drop NaNs
# df = df.dropna(subset=['JobFactors'], axis=0)


In [ ]:
JobFactors_strs


**ENCODE JOBFACTORS**

In [ ]:
# https://stackoverflow.com/questions/54757552/how-to-add-dummies-to-pandas-dataframe

# get one hot encoding of non-ordinal feature column
one_hot_job_factors = pd.get_dummies(df[['JobFactors']])


# drop feature column & add encoded features
## df = df.drop('JobFactors', axis=1)
# join encoded feature to df
##df = df.join(one_hot_job_factors)

one_hot_job_factors.head(3)


In [ ]:
# check expanded feature df
print('df with features:', df.shape)


In [ ]:
df.shape

# JobSeek

In [ ]:
# drop NaNs
df = df.dropna(subset=['JobSeek'], axis=0)


In [ ]:
# jobseek encoding dictionary
JobSeek_dict = {'I am actively looking for a job': 0,
                 'I’m not actively looking, but I am open to new opportunities': 1,
                 'I am not interested in new job opportunities': 2}


In [ ]:
# encode JobSeek as integer
df['JobSeek_encode'] = replace_values_in_string(df.JobSeek, JobSeek_dict)


In [ ]:
df['JobSeek_encode']


In [ ]:
df.shape

# WorkWeekHrs
* remove NaNs
* impute NaNs

In [ ]:
# drop NaN WorkWeekHrs
df = df.dropna(subset=['WorkWeekHrs'], axis=0)


In [ ]:
df.shape


# YearsCode

In [ ]:
# drop nulls Years Coding responses
df = df[df['YearsCode'].notnull()]


In [ ]:
# replace 'Less than 1 year' with 0
# replace 'More than 50 years' with 51
df['YearsCode'] = df['YearsCode'].str.replace('Less than 1 year', '0')
df['YearsCode'] = df['YearsCode'].str.replace('More than 50 years', '51')


In [ ]:
# convert 'Age1stCode' to type integer
df['YearsCode'] = df['YearsCode'].astype(str).astype(int)


In [ ]:
df['YearsCode'].head()


In [ ]:
df.shape


# YearsCodePro

In [ ]:
# drop nulls Years Coding responses
df = df[df['YearsCodePro'].notnull()]


In [ ]:
# replace 'Less than 1 year' with 0
# replace 'More than 50 years' with 51
df['YearsCodePro'] = df['YearsCodePro'].str.replace('Less than 1 year', '0')
df['YearsCodePro'] = df['YearsCodePro'].str.replace('More than 50 years', '51')


In [ ]:
# convert 'Age1stCode' to type integer
df['YearsCodePro'] = df['YearsCodePro'].astype(str).astype(int)


In [ ]:
df['YearsCodePro'].head()


In [ ]:
df.shape


RESET INDEX

In [ ]:
# reset index
df = df.reset_index(drop=True)


PRUNE DATASET FOR MODELING 


In [ ]:
list(df.columns)


In [ ]:
df.head()

In [ ]:
# drop original feature columns
# ordinal encoding of 'JobSat' is dropped
df.drop(['Hobbyist', 'EdLevel', 'JobSat','JobFactors','JobSeek', 'JobSat_encode'], axis=1, inplace=True)


In [ ]:
df.shape


In [ ]:
interrupt before overwrite